In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
     model_name='llama-3.1-70b-versatile',
    temperature=0,
    groq_api_key='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'
   
)
response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-41366?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer I

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(

    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: 'role','experience','skills',and 'description'.
    Only return the valid JSON
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input= {'page_data':page_data})
print(res.content)

```json
{
  "role": "Software Engineer I",
  "experience": "0-3+ years",
  "skills": [
    "Solid computer science background with a focus on patterns and algorithmic complexity",
    "Able to quickly pick up new development paradigms, technologies, and frameworks",
    "Solid programming skills in at least one programming language like Java, Scala, Python, JavaScript, Golang, or other programming language",
    "Experience with SQL",
    "Strong verbal and written communication and collaboration skills"
  ],
  "description": "We are seeking a Software Engineer I to help build tooling to support and drive the Privacy and Data Governance initiatives within Global Technology at Nike. This Software Engineer will be expected to execute on the technical implementation of new features, provide support for existing products and services as well as coordinate with non-technical teammates."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I',
 'experience': '0-3+ years',
 'skills': ['Solid computer science background with a focus on patterns and algorithmic complexity',
  'Able to quickly pick up new development paradigms, technologies, and frameworks',
  'Solid programming skills in at least one programming language like Java, Scala, Python, JavaScript, Golang, or other programming language',
  'Experience with SQL',
  'Strong verbal and written communication and collaboration skills'],
 'description': 'We are seeking a Software Engineer I to help build tooling to support and drive the Privacy and Data Governance initiatives within Global Technology at Nike. This Software Engineer will be expected to execute on the technical implementation of new features, provide support for existing products and services as well as coordinate with non-technical teammates.'}

In [17]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [21]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [31]:
links = collection.query(query_texts=["Experience in Python","Expertise in React"],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [33]:
job = json_res
job['skills']

['Solid computer science background with a focus on patterns and algorithmic complexity',
 'Able to quickly pick up new development paradigms, technologies, and frameworks',
 'Solid programming skills in at least one programming language like Java, Scala, Python, JavaScript, Golang, or other programming language',
 'Experience with SQL',
 'Strong verbal and written communication and collaboration skills']

In [35]:
prompt_email = PromptTemplate.from_template(

    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfill their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job),"link_list":links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's Privacy and Data Governance Initiatives

Dear Hiring Manager,

I came across the Software Engineer I role at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals in building tooling to support and drive the Privacy and Data Governance initiatives within Global Technology.

At AtliQ, we have a proven track record of delivering high-quality software solutions that meet the needs of our clients. Our team of expert software engineers possesses a solid computer science background, with a focus on patterns and algorithmic complexity. We're well-versed in various programming languages, including Java, Scala, Python, JavaScript, and Golang, and have extensive experience with SQL.

Our expertise in software development, combined with our ability to quickly adapt to new technologies and frameworks, makes us an ideal partner for Nike's initiatives. We're confident that our team 